# 设定路径

In [2]:
def setProjectPath(projectPath):
    import os
    import sys
    root=os.path.abspath(projectPath)
    print(root)
    sys.path.append(root)
    os.environ["TORCH_HOME"]=r"E:\Data\torch-model"
    import warnings  
    warnings.filterwarnings("ignore", category=RuntimeWarning, module="mne")

projectPath=r'D:\project_meta\NNproject\NNI'
setProjectPath(projectPath)
import mne
datasetFolder=r'D:\dataset\driver_dataset\DROZY\DROZY'
outputFolder=r'D:\project_meta\NNproject\NNI\output'
from project.dataprocess.FolderTree import *
datasetDict=getDataPath(datasetFolder)
outputDict=getOutPath(outputFolder)
import pprint
print('datasetfolder:')
pprint.pprint(datasetDict)
print('outputfolder:')
pprint.pprint(outputDict)


D:\project_meta\NNproject\NNI
datasetfolder:
{'annotation_auto_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\annotations-auto',
 'annotation_manual_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\annotations-manual',
 'kinect_file': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\kinect-intrinsics.yaml',
 'kss_file': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\KSS.txt',
 'psg_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\psg',
 'pvt_rt_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\pvt-rt',
 'videos_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\videos_i8'}
outputfolder:
{'ECG_table_file': 'D:\\project_meta\\NNproject\\NNI\\output\\ECG_table.csv',
 'EEG_table_file': 'D:\\project_meta\\NNproject\\NNI\\output\\EEG_table.csv',
 'bandpower_file': 'D:\\project_meta\\NNproject\\NNI\\output\\bandpower.csv',
 'cwt_folder': 'D:\\project_meta\\NNproject\\NNI\\output\\cwt',
 'filted_folder': 'D:\\project_meta\\NNproject\\NNI\\output\\psg_filted',
 'info_file': 'D:\\proj

In [3]:
import os
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [4]:
IDFrameFolder=os.path.join(outputDict['video_frames'],'IDFolder')
print(IDFrameFolder)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整图片大小
    transforms.ToTensor(),  # 转换为张量
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 归一化
])

dataset=ImageFolder(IDFrameFolder,transform)


D:\project_meta\NNproject\NNI\output\video_frames\IDFolder


In [14]:
person_foler=[os.path.join(IDFrameFolder,str(person)) for person in range(1,14)]
print(person_foler)
test_dataset = torch.utils.data.Subset(dataset, person_foler[0])


['D:\\project_meta\\NNproject\\NNI\\output\\video_frames\\IDFolder\\1', 'D:\\project_meta\\NNproject\\NNI\\output\\video_frames\\IDFolder\\2', 'D:\\project_meta\\NNproject\\NNI\\output\\video_frames\\IDFolder\\3', 'D:\\project_meta\\NNproject\\NNI\\output\\video_frames\\IDFolder\\4', 'D:\\project_meta\\NNproject\\NNI\\output\\video_frames\\IDFolder\\5', 'D:\\project_meta\\NNproject\\NNI\\output\\video_frames\\IDFolder\\6', 'D:\\project_meta\\NNproject\\NNI\\output\\video_frames\\IDFolder\\7', 'D:\\project_meta\\NNproject\\NNI\\output\\video_frames\\IDFolder\\8', 'D:\\project_meta\\NNproject\\NNI\\output\\video_frames\\IDFolder\\9', 'D:\\project_meta\\NNproject\\NNI\\output\\video_frames\\IDFolder\\10', 'D:\\project_meta\\NNproject\\NNI\\output\\video_frames\\IDFolder\\11', 'D:\\project_meta\\NNproject\\NNI\\output\\video_frames\\IDFolder\\12', 'D:\\project_meta\\NNproject\\NNI\\output\\video_frames\\IDFolder\\13']


In [16]:
import os
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch
# 数据集根目录
data_dir = IDFrameFolder

# 定义数据转换
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整图片大小
    transforms.ToTensor(),  # 转换为张量
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 归一化
])

# 加载数据集
dataset = ImageFolder(root=data_dir, transform=transform)

# 获取每个人的文件夹路径
person_folders = [os.path.join(data_dir, person) for person in os.listdir(data_dir)]

# 根据需要指定测试集和训练集
test_persons = [2, 5, 9]  # 假设选择第2、第5和第9个人作为测试集
train_persons = [idx for idx in range(0, 14) if idx not in test_persons]  # 其他人作为训练集

# 创建测试集和训练集的数据加载器
test_dataset = torch.utils.data.Subset(dataset, [person_folders[person] for person in test_persons])
train_dataset = torch.utils.data.Subset(dataset, [person_folders[person] for person in train_persons])

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 打印数据集信息
print(f"总样本数： {len(dataset)}")
print(f"测试集样本数：{len(test_dataset)}")
print(f"训练集样本数：{len(train_dataset)}")

总样本数： 3305
测试集样本数：3
训练集样本数：11


In [23]:
train_persons

[0, 1, 3, 4, 6, 7, 8, 10, 11, 12, 13]

In [31]:
from sklearn.model_selection import KFold

personId=list(range(1,15))

kf = KFold(n_splits=7, shuffle=True, random_state=42)
for kt_i,(train_persons,test_persons) in enumerate(kf.split(person_folders)):
    # test_persons = [2, 5, 9]  # 假设选择第2、第5和第9个人作为测试集
    # train_persons = [idx for idx in range(0, 14) if idx not in test_persons]  # 其他人作为训练集

    # 创建测试集和训练集的数据加载器
    test_dataset = torch.utils.data.Subset(dataset, [person_folders[person] for person in test_persons])
    train_dataset = torch.utils.data.Subset(dataset, [person_folders[person] for person in train_persons])

    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    # 打印数据集信息
    print(f"总样本数： {len(dataset)}")
    print(f"测试集样本数：{len(test_dataset)}")
    print(f"训练集样本数：{len(train_dataset)}")


总样本数： 3305
测试集样本数：2
训练集样本数：12
总样本数： 3305
测试集样本数：2
训练集样本数：12
总样本数： 3305
测试集样本数：2
训练集样本数：12
总样本数： 3305
测试集样本数：2
训练集样本数：12
总样本数： 3305
测试集样本数：2
训练集样本数：12
总样本数： 3305
测试集样本数：2
训练集样本数：12
总样本数： 3305
测试集样本数：2
训练集样本数：12


In [6]:
import os
import random
import numpy as np
from collections import defaultdict
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold
import torch
# 数据集根目录
data_dir =  IDFrameFolder

# 定义数据转换
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整图片大小
    transforms.ToTensor(),  # 转换为张量
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 归一化
])

# 加载数据集
dataset = ImageFolder(root=data_dir, transform=transform)

# 获取每个人的文件夹路径及人的编号
person_folders = [(os.path.join(data_dir, person), person) for person in os.listdir(data_dir)]

# 将人的索引作为字典的键，将每个人的图片路径保存在对应的列表中
person_images = defaultdict(list)
for folder, person in person_folders:
    filenames = os.listdir(folder)
    for filename in filenames:
        if filename.endswith('.jpg') or filename.endswith('.png'):
            path = os.path.join(folder, filename)
            person_images[person].append(path)

# 以人为单位划分K折
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 分割索引
train_indices = []
test_indices = []
for _, test_persons in kf.split(person_folders):
    # 选中测试集的人的编号
    test_person_ids = [person_folders[i][1] for i in test_persons]
    # 将每个人的图片按照选中的测试集和训练集划分
    train_files = []
    test_files = []
    for person, images in person_images.items():
        if person in test_person_ids:
            test_files.extend(images)
        else:
            train_files.extend(images)
    # 记录训练集和测试集的索引
    train_indices.append([dataset.index(x) for x in train_files])
    test_indices.append([dataset.index(x) for x in test_files])

# 创建数据加载器
train_loaders = []
test_loaders = []
for i in range(5):
    train_sampler = torch.utils.data.SubsetRandomSampler(train_indices[i])
    test_sampler = torch.utils.data.SubsetRandomSampler(test_indices[i])
    train_loader = DataLoader(dataset, batch_size=32, sampler=train_sampler)
    test_loader = DataLoader(dataset, batch_size=32, sampler=test_sampler)
    train_loaders.append(train_loader)
    test_loaders.append(test_loader)

# 打印数据集和交叉验证信息
print(f"总样本数：{len(dataset)}")
print(f"{kf.n_splits}折交叉验证")
for i in range(kf.n_splits):
    print(f"第{i+1}次验证集大小：{len(test_indices[i])}")

总样本数：3305
5折交叉验证
第1次验证集大小：0
第2次验证集大小：0
第3次验证集大小：0
第4次验证集大小：0
第5次验证集大小：0
